# Graveyard

In [ ]:
# TAKEN FROM ASSETEXPAND CLASS. REPLACED WITH VECTORIZE FOR PROCESSING SPEED
def to_vec(self):

        # returns clean series with lists of assets. 

        vec = self.data[self.col].apply(lambda x: re.sub(r'[{}\']', '', x))
        # remove extra spaces
        vec = vec.apply(lambda x: x.replace(" ", ""))
        # a bit of a band aid. str.split was throwing errors when added above
        # print(vec.head(2))
        return pd.Series([i.split(',') for i in vec])



In [ ]:
# ASSET FEATURE EXPANSION, MY ORIGINAL VERSION - (SO SLOW)
import re
asset_col = news_train['assetCodes'].apply(lambda x: re.sub(r'[{}\']', '', x).split(','))

df_idx = pd.DataFrame()
for i in range(0,asset_col.size):
    index = asset_col.index[i]
    for j in range(0, len(asset_col[i])):
        df_idx = pd.concat([df_idx, pd.Series([asset_col[i][j]], 
                                              index=[index])], axis=0)

df_news = pd.concat([df_idx, news_train], axis=1,
                 join='outer',join_axes=[df_idx.index]).reset_index(drop=True) 
                                                                    
df_news.drop(['assetCodes'], axis=1, inplace=True)

df_news.rename(columns={0:'assetCode'}, inplace=True)


In [ ]:
# FROM EDA FORK. PROTOTYPE FUNC

# disagree with drops
def preprocess_news(news_train):
    drop_list = [
        'audiences', 'subjects', 'assetName',
        'headline', 'firstCreated', 'sourceTimestamp',
    ]
    news_train.drop(drop_list, axis=1, inplace=True)
    
    # Factorize categorical columns
    for col in ['headlineTag', 'provider', 'sourceId']:
        news_train[col], uniques = pd.factorize(news_train[col])
        del uniques
    
    # Remove {} and '' from assetCodes column
    news_train['assetCodes'] = news_train['assetCodes'].apply(lambda x: x[1:-1].replace("'", ""))
    return news_train

news_train = preprocess_news(news_train)

In [ ]:
# MORE ASSET EXPANSION CODE
# THIS WASN'T USED, BUT IT COULD HAVE BEEN
sub = df_news.subjects.apply(lambda x: re.sub(r'[{}\']', '', x)).replace(" ", "")
lst = []
for i in sub:
    val = i.split()
    lst.extend(val)
    
col_names = list(set([i.replace(',', '') for i in lst]))
col_names = list(set([i.strip(' \n,') for i in lst]))


In [ ]:
# Tried to dynamically import

def import_class(modulename, classname):
    ''' Returns imported class. '''
    try:
        return getattr(__import__(modulename, globals(), locals(), [classname], -1), classname)
    except AttributeError:
        print('Error in importing class. "%s" has no class "%s"' % (modulename, classname))
        return None
    except ImportError as e:
        print('Error in importing class: %s' % (e))
        return None

mymodule = import_class("sigma_libs", "*")    

In [ ]:
import importlib

from importlib.import_module("sigma_libs") import *

mymodule = importlib.import_module('matplotlib.text')
imported_module = __import__(file) 


In [ ]:
# still trying vectorize
def vec_func(x):
    if x <= 0.25:
        return 1
    else:
        return 0

vf = np.vectorize(vec_func, cache=False)
data['Bollinger_bearish'] = vf(data['BB_lower_dis']).astype('bool')
data['Bollinger_bullish'] = vf(data['BB_upper_dis']).astype('bool')


# Extra - possible use

In [ ]:
# BACKTRADER
'''backtrader looks interesting. May be worth comparing to 
Quandle or zipline'''
class SmaCross(bt.SignalStrategy):
        params = (('pfast', 10), ('pslow', 30),)
        def __init__(self):
            sma1, sma2 = bt.ind.SMA(period=self.p.pfast), bt.ind.SMA(period=self.p.pslow)
            self.signal_add(bt.SIGNAL_LONG, bt.ind.CrossOver(sma1, sma2))

cerebro = bt.Cerebro()

data = bt.feeds.YahooFinanceData(dataname='MSFT', fromdate=datetime(2016, 1, 1),
                                 todate=datetime(2019, 1, 31))

# help(data.array)

cerebro.adddata(data)

cerebro.addstrategy(SmaCross)
cerebro.run()
cerebro.plot()

In [ ]:
# SOME PLOTTING CODE TO GO THROUGH  
set([asset for asset in df_market.assetCode if re.match(r'AMZ', asset)])

amzn_jan = df_market[(df_market.time.dt.year == 2007) & 
                      (df_market.assetCode == 'AMZN.O')]

plt.figure(figsize=(10,6))
# plt.plot(range(len(aapl_jan.time)), aapl_jan.close, label='Close price')
# plt.plot(range(len(aapl_jan.time)), aapl_jan.open, label='Open price')
plt.title("Opening and closing price")
plt.plot(amzn_jan.time, amzn_jan.open, label='Open price')
plt.plot(amzn_jan.time, amzn_jan.close, label='Close price')
plt.legend()
plt.show()

plt.figure(figsize=(10,6))
plt.title("Opening and closing return mtres 1")
plt.bar(range(len(amzn_jan.time)), amzn_jan.returnsOpenPrevMktres1, label='Return Open price')
plt.bar(range(len(amzn_jan.time)), amzn_jan.returnsClosePrevMktres1, label='Return Close price')
plt.legend()
plt.show()

# LOG RETURNS
amzn_daily_return = np.log(amzn_jan.close/ amzn_jan.close.shift(1))

# NON-LOG
amzn_daily_return_non = amzn_jan.close/ amzn_jan.close.shift(1) - 1
pd.concat([amzn_daily_return, amzn_daily_return_non], axis=1).head()
# CLOSE BUT DEFINITELY SLIGHTLY DIFFERENT
amzn_daily_return.hist(bins=50)

# SOME NANS - BUT A VERY LOW PERCENT. PROBABLY FROM THE SHIFT, WHICH IS 
# WHY THE NANS ARE FRONT LOADED - SEEMINGLY

df_market = df_market.assign(
    daily_percent_price=df_market.groupby('assetCode',
                                            as_index=False).apply(lambda x: np.log(x.close/ x.close.shift(1)))
    .reset_index(0, drop=True)
)

# df_market.daily_percent_price.isnull().sum() / df_market.daily_percent_price.size

plt.figure(figsize=(12,8))
ax1 = plt.subplot(221)
df_market.query("time.dt.year == 2016 and assetCode == 'AAPL.O'")['daily_percent_price'].hist(bins=50)
ax2 = plt.subplot(222)
df_market.query("time.dt.year == 2016 and assetCode == 'AMZN.O'")['daily_percent_price'].hist(bins=50)
ax3 = plt.subplot(223)
df_market.query("time.dt.year == 2016 and assetCode == 'A.N'")['daily_percent_price'].hist(bins=50)
ax4 = plt.subplot(224)
df_market.query("time.dt.year == 2016 and assetCode == 'CMC.N'")['daily_percent_price'].hist(bins=50)
plt.show()

In [ ]:
# INTERESTING GROUPBY METHOD
# NEED TO RUN THE DATE FUNC FIRST
df_g = (
    df.
    reset_index().
    sort_values(['asset', 'date']).
    set_index(['asset','date'])
)

test = df.groupby('asset')

for name,group in test:
    print(name)
    print(group)

0.000000 - -0.075508

(df.
    groupby('asset').
    apply(lambda x: x.returns_close_raryw - x.returns_open_raw10).
    reset_index(0, drop=True)
)

## Old functions

In [ ]:
# Cool convolution
N=20
np.convolve(qcom, np.ones((N,))/N, mode='valid')[-5:]

In [ ]:
# ATTEMPT TO VECTORIZE - Multi-indexing was really slow

mult = csv.copy()
mult.set_index(['assetCode', 'time'], inplace=True)
mult.sort_index(inplace=True)

sma_v = np.array([])

sma = np.zeros(df.shape[0])
for asset in assets:
    print(asset)
    X = mult['close'].loc[asset].values
    x = np.arange(X.shape[0])
    def sma(i, n=20):
        return np.nansum(X[i-n+1:i+1]) / n
    vf = np.vectorize(sma)
    # print(vf(x)[-5:])
    arr = vf(x)
    sma_v = np.concatenate((sma_v, arr))
    
    